There are 541 rows in the seismic data sets，the first row is the seismic monitor working site id, the second to the 13h row are some characters, the others are time series. First, we code those working site id as follow.

In [1]:
import tensorflow as tf
import h5py

Load the preprocessed data.

In [2]:
file = h5py.File('E:\\数据\\AAIA 16\\Pre_combined.h5', 'r')
data = file['train_data'][:]
label = file['train_label'][:]
test_data = file['test_data'][:]
test_label = file['test_label'][:]
file.close()

In [3]:
sess=tf.InteractiveSession()
embedding=tf.get_variable("embedding", [data.shape[0],10])
input_ids=tf.placeholder(dtype=tf.int32,shape=[None])
input_embedding=tf.nn.embedding_lookup(embedding,input_ids)
tf.global_variables_initializer().run()
site_embedding=sess.run(input_embedding,feed_dict={input_ids:data[:,0]})

In [4]:
print (site_embedding.shape)

(133151, 10)


In [14]:
file1 = h5py.File('site_embedding_raw.h5', 'w')
file1.create_dataset('site_embedding_raw', data=site_embedding.reshape([-1, 10]))
file1.close()

After coding, we got the [133151,10] embedding working site id. Namely,133151 samples and their dimension is 10.

In [5]:
import model
import data_pre
import numpy as np

For the time series, we consider to use the RNN model to abstract some features.

In [6]:
# hyperparameters
batch_size = 128
n_inputs = 22 # 28 cl
n_steps = 24 # 28 rows -> time stamps
n_classes = 2
epochs = 40

In [7]:
data_raw = data_pre.DataSet(np.array(data[:, 13:]), np.array(label))
test = data_pre.DataSet(np.array(test_data[:,13:]), np.array(test_label))

RNN model is in the file model.py

In [8]:
xs = tf.placeholder(tf.float32, [None, n_steps, n_inputs], name="inputs")
ys = tf.placeholder(tf.float32, [None, n_classes], name="outputs")
model = model.RNN_Model(xs, ys)

In [11]:
    with tf.Session() as sess:
        writer = tf.summary.FileWriter('logs/', sess.graph)
        init = tf.global_variables_initializer()
        sess.run(init)
        batch = data_raw.num_examples / batch_size
        # learning_rate=0.0005
        for epoch in range(epochs):
            for i in range(int(batch)):
                batch_x, batch_y = data_raw.next_batch(batch_size)
                batch_x = batch_x.reshape([batch_size, n_steps, n_inputs])
                sess.run(model.optimizer, feed_dict={xs: batch_x, ys: batch_y})
            accu, loss = sess.run(model.accuracy, feed_dict={xs: batch_x, ys: batch_y})
            print('epoch:', epoch + 1, 'loss:', loss, 'train accuracy:', accu)
        outpus=sess.run(model.predict[1], feed_dict={xs: data_raw.images.reshape([-1, n_steps, n_inputs]), ys: data_raw.labels})

epoch: 1 loss: 1.56642 train accuracy: 0.976563
epoch: 2 loss: 1.13435 train accuracy: 0.984375
epoch: 3 loss: 0.855604 train accuracy: 0.984375
epoch: 4 loss: 0.610998 train accuracy: 0.96875
epoch: 5 loss: 0.401953 train accuracy: 0.984375
epoch: 6 loss: 0.305275 train accuracy: 0.984375
epoch: 7 loss: 0.270464 train accuracy: 0.945313
epoch: 8 loss: 0.128442 train accuracy: 0.984375
epoch: 9 loss: 0.103542 train accuracy: 0.984375
epoch: 10 loss: 0.0993395 train accuracy: 0.984375
epoch: 11 loss: 0.11287 train accuracy: 0.976563
epoch: 12 loss: 0.174547 train accuracy: 0.953125
epoch: 13 loss: 0.102404 train accuracy: 0.976563
epoch: 14 loss: 0.120614 train accuracy: 0.976563
epoch: 15 loss: 0.0253493 train accuracy: 1.0
epoch: 16 loss: 0.0622635 train accuracy: 0.992188
epoch: 17 loss: 0.0550989 train accuracy: 0.984375
epoch: 18 loss: 0.110551 train accuracy: 0.96875
epoch: 19 loss: 0.124411 train accuracy: 0.976563
epoch: 20 loss: 0.0758985 train accuracy: 0.984375
epoch: 21 loss

save the extracted features

In [12]:
file1 = h5py.File('result_raw.h5','w')
file1.create_dataset('result',data=outpus.reshape([-1,1200]))#data shape is [3778656,50]
file1.close()

Check the saved files

In [15]:
file = h5py.File('result_raw.h5','r')
seq_raw = file['result'][:]
file.close()

file = h5py.File('site_embedding_raw.h5','r')
work_id_raw = file['site_embedding_raw'][:]
file.close()

In [16]:
cha_raw = data[:,1:13]

Combine those features.

In [17]:
train_raw_features=np.concatenate((work_id_raw,cha_raw,seq_raw),axis=1)

Save them.

In [19]:
file1 = h5py.File('train_features_raw.h5','w')
file1.create_dataset('train_raw_features',data=train_raw_features)
file1.close()

file1 = h5py.File('test_features.h5','r')
x_test=file1['test_features'][:]
file1.close()


file = h5py.File('features_raw_combine.h5','w')
file.create_dataset('train_raw_features',data=train_raw_features)
file.create_dataset('train_labels',data=label)
file.create_dataset('test_features',data=x_test)
file.create_dataset('test_labels',data=test_label)
file.close()